In [99]:
import psycopg2

### Sample postgres table:

```
CREATE TABLE edge_table (
    id BIGSERIAL,
    source BIGINT,
    target BIGINT,
    cost FLOAT,
    x1 FLOAT,
    y1 FLOAT,
    x2 FLOAT,
    y2 FLOAT
);
```

### Postgres connection

In [100]:
conn = psycopg2.connect(
    host="localhost",
    database="krakow",
    user="postgres",
    password="postgres")

### Data loading

In [101]:
conn.autocommit = True
cursor = conn.cursor() 

In [106]:
drop_table = "DROP TABLE IF EXISTS edge_table CASCADE;"

create_table = """
    CREATE TABLE edge_table (
    id BIGSERIAL,
    osmid BIGINT,
    source BIGINT,
    target BIGINT,
    length FLOAT,
    name VARCHAR,
    highway VARCHAR,
    x1 FLOAT,
    y1 FLOAT,
    x2 FLOAT,
    y2 FLOAT);
"""

create_extension = """
    CREATE EXTENSION IF NOT EXISTS pgrouting CASCADE;
    CREATE EXTENSION IF NOT EXISTS postgis;
"""

create_index = """
    CREATE INDEX source_idx ON edge_table (source);
    CREATE INDEX target_idx ON edge_table (target);
"""

### Drop table (if exists)

In [107]:
cursor.execute(drop_table)

### Create extensions (pgRouting and postgis)

In [108]:
cursor.execute(create_extension)

### Create table

In [109]:
cursor.execute(create_table)

### Load data from csv

In [110]:
# TODO: remember to change path to .csv file
csv_file_name = "/home/pcend/Piotr/kod/lisboa-routing/data/krakow/edge_table_sql.csv"
copy_csv_query = "COPY edge_table(source, target, osmid, length, name, highway, x1, y1, x2, y2) FROM STDIN DELIMITER ',' CSV HEADER"

In [111]:
cursor.copy_expert(copy_csv_query, open(csv_file_name, "r"))

### Create indexes on source and target

In [112]:
cursor.execute(create_index)

### Search queries

In [113]:
ids = (356926768, 2104495834) # short (zakamycze, betel)
# ids = (13822575, 11286442329) # random small

In [137]:
dijkstra = f"""
    SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, length as cost, length as reverse_cost FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

dijkstra_benchmark = f"""
    EXPLAIN ANALYZE 
    SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, length as cost, length as reverse_cost FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

astar = f"""
    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

astar_benchmark = f"""
    EXPLAIN ANALYZE
    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

In [138]:
print(astar)
print(dijkstra)


    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 FROM edge_table',
    356926768, 2104495834,
    directed => false);


    SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, length as cost, length as reverse_cost FROM edge_table',
    356926768, 2104495834,
    directed => false);



In [133]:
cursor.execute(astar)

result = cursor.fetchall()

In [134]:
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['seq', 'path_seq', 'node', 'edge', 'cost', 'agg_cost']
(1, 1, 356926768, 121985902, 40.151, 0.0)
(2, 2, 356926768, 121985902, 40.151, 40.151)
(3, 3, 356926768, 121985902, 22.495, 80.302)
(4, 4, 356926768, 121985902, 22.495, 102.79700000000001)
(5, 5, 2583143012, 121985902, 22.495, 125.29200000000002)
(6, 6, 2583143012, 121985902, 22.495, 147.787)
(7, 7, 2583143012, 121985902, 30.134, 170.282)
(8, 8, 2583143012, 121985902, 30.134, 200.416)
(9, 9, 2583143013, 252223476, 16.071, 230.55)
(10, 10, 2583143013, 252223476, 16.071, 246.621)
(11, 11, 1364114321, 252223476, 16.071, 262.692)
(12, 12, 1364114321, 252223476, 7.768, 278.76300000000003)
(13, 13, 1364114321, 252223476, 7.768, 286.531)
(14, 14, 1364114321, 252223476, 16.071, 294.299)
(15, 15, 2000518505, 252223476, 13.455, 310.37)
(16, 16, 2000518505, 252223476, 7.768, 323.825)
(17, 17, 2000518505, 252223476, 7.768, 331.59299999999996)
(18, 18, 2000518505, 252223476, 13.455, 339.36099999999993)
(19, 19, 356926771, 252223476, 13.455, 35

In [135]:
cursor.execute(dijkstra)

result = cursor.fetchall()

In [136]:
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['seq', 'path_seq', 'start_vid', 'end_vid', 'node', 'edge', 'cost', 'agg_cost']
(1, 1, 356926768, 2104495834, 356926768, 121985902, 22.495, 0.0)
(2, 2, 356926768, 2104495834, 2583143012, 121985902, 30.134, 22.495)
(3, 3, 356926768, 2104495834, 2583143013, 252223476, 16.071, 52.629000000000005)
(4, 4, 356926768, 2104495834, 1364114321, 252223476, 7.768, 68.7)
(5, 5, 356926768, 2104495834, 2000518505, 252223476, 13.455, 76.468)
(6, 6, 356926768, 2104495834, 356926771, 252223476, 31.201, 89.923)
(7, 7, 356926768, 2104495834, 356926775, 252223476, 21.388, 121.124)
(8, 8, 356926768, 2104495834, 2583143014, 252223476, 15.735, 142.512)
(9, 9, 356926768, 2104495834, 2583143015, 252223476, 27.355, 158.247)
(10, 10, 356926768, 2104495834, 1752636499, 252222714, 14.562, 185.602)
(11, 11, 356926768, 2104495834, 2583137150, 252222714, 12.532, 200.16400000000002)
(12, 12, 356926768, 2104495834, 2583137151, 252222714, 13.823, 212.69600000000003)
(13, 13, 356926768, 2104495834, 2583137152, 252222714, 

### Disconnect

In [82]:
conn.commit() 
conn.close() 